In [10]:
from pyspark import SparkContext
import os

# Set parent directory

In [18]:
base = os.path.abspath(os.pardir)
data_dir = os.path.join(base, "work", "files")
data_dir

'/home/jovyan/work/files'

# Spark Context

In [4]:
sc = SparkContext(master="local", appName="trasnformacionesYAacciones")

In [7]:
sc

<SparkContext master=local appName=trasnformacionesYAacciones>

In [5]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [6]:
rdd1.collect() # Don't use...

[1, 2, 3]

In [19]:
equipos_olimpicos_rdd = sc.textFile(f"{data_dir}/paises.csv").map(lambda line: line.split(","))

In [21]:
equipos_olimpicos_rdd.take(5)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

In [22]:
sc.stop()